In [15]:
import pandas as pd
import os


# Expression data section

In [46]:
def extract_species_id_and_column_name(column_name, reference_columns):
    parts = column_name.split('_')
    if '_'.join(parts[2:]) in reference_columns:
        return '_'.join(parts[:2]), '_'.join(parts[2:])
    else:
        return parts[0], '_'.join(parts[1:])

def read_xlsx_files_to_dataframe(directory, concatenated_df=None):
    # Initialize an empty dataframe if concatenated_df is not provided
    if concatenated_df is None:
        concatenated_df = pd.DataFrame()
    
    # Get the set of species that have already been read
    existing_species = set(concatenated_df['Species']) if 'Species' in concatenated_df.columns else set()
    
    # Reference columns for species ID extraction
    reference_columns = {col.split('_')[0] for col in concatenated_df.columns if 'TPM' in col}
    reference_columns.update({'_'.join(col.split('_')[:2]) for col in concatenated_df.columns if 'TPM' in col})
    
    # Iterate over all files in the given directory
    for filename in os.listdir(directory):
        if filename.endswith(".xlsx"):
            file_path = os.path.join(directory, filename)
            
            # Remove the extension from the filename
            base_filename = os.path.splitext(filename)[0]
            
            # Read the Excel file into a pandas dataframe
            df = pd.read_excel(file_path)
            
            # Check if the species in the file is already read
            if 'Species' in df.columns:
                species_in_file = set(df['Species'])
                if species_in_file.intersection(existing_species):
                    print(f"Species {species_in_file} is skipped as it has already been read from {base_filename}.")
                    continue
            
            # Extract 'Species' and 'Region' columns
            columns_to_extract = ['Species', 'Chromosome', 'Region']
            
            # Identify TPM columns and extract them
            tpm_columns = [col for col in df.columns if 'TPM' in col]
            
            # Create a new dataframe with the required columns
            extracted_data = df[columns_to_extract + tpm_columns].copy()
            
            # Extract species id from the TPM columns and rename them
            species_ids = []
            for col in tpm_columns:
                species_id, new_col_name = extract_species_id_and_column_name(col, reference_columns)
                extracted_data[new_col_name] = extracted_data[col]
                extracted_data.drop(columns=[col], inplace=True)
                species_ids.append(species_id)
            
            try:
                if len(set(species_ids)) == 1:
                    extracted_data['Species ID'] = species_ids[0]
                else:
                    raise ValueError("Multiple species IDs found in one file.")
            except ValueError as e:
                print(f"Error processing file {filename}: {e}")
                extracted_data['Species ID'] = species_ids[0]

            # Concatenate the extracted data to the main dataframe
            if concatenated_df.empty:
                concatenated_df = extracted_data
            else:
                if set(concatenated_df.columns) == set(extracted_data.columns):
                    concatenated_df = pd.concat([concatenated_df, extracted_data], ignore_index=True)
                    print(f"Data from {filename} is extracted")

                else:
                    unmatched_columns = set(concatenated_df.columns).symmetric_difference(set(extracted_data.columns))
                    print(f"Column names do not match across the files. Unmatched columns: {unmatched_columns}")
                    return concatenated_df
    
    return concatenated_df


In [17]:
# Reads expression data and creates a dataframe. 
directory = 'data_expression'
final_df = read_xlsx_files_to_dataframe(directory)
print(final_df)

ACHX As_1 (GE) - TPM
ACHX As_2 (GE) - TPM
ACHX As_3 (GE) - TPM
ACHX Bs_1 (GE) - TPM
ACHX Bs_2 (GE) - TPM
ACHX Bs_3 (GE) - TPM
ACHX Ctrl_1 (GE) - TPM
ACHX Ctrl_2 (GE) - TPM
ACHX Ctrl_3 (GE) - TPM
ACHX Li_1 (GE) - TPM
ACHX Li_2 (GE) - TPM
ACHX Li_3 (GE) - TPM
ACHX Mig_1 (GE) - TPM
ACHX Mig_2 (GE) - TPM
ACHX Mig_3 (GE) - TPM
ACHX Nd_1 (GE) - TPM
ACHX Nd_2 (GE) - TPM
ACHX Nd_3 (GE) - TPM
ACHX Ns_1 (GE) - TPM
ACHX Ns_2 (GE) - TPM
ACHX Ns_3 (GE) - TPM
ACHX Oss_1 (GE) - TPM
ACHX Oss_2 (GE) - TPM
ACHX Oss_3 (GE) - TPM
ACHX Oxs_1 (GE) - TPM
ACHX Oxs_2 (GE) - TPM
ACHX Oxs_3 (GE) - TPM
ACHX Sp_1 (GE) - TPM
ACHX Sp_2 (GE) - TPM
ACHX Sp_3 (GE) - TPM
ACHX Tm_1 (GE) - TPM
ACHX Tm_2 (GE) - TPM
ACHX Tm_3 (GE) - TPM
ACHX Vic_1 (GE) - TPM
ACHX Vic_2 (GE) - TPM
ACHX Vic_3 (GE) - TPM
ACIB As_1 (GE) - TPM
ACIB As_2 (GE) - TPM
ACIB As_3 (GE) - TPM
ACIB Bs_1 (GE) - TPM
ACIB Bs_2 (GE) - TPM
ACIB Bs_3 (GE) - TPM
ACIB Ctrl_1 (GE) - TPM
ACIB Ctrl_2 (GE) - TPM
ACIB Ctrl_3 (GE) - TPM
ACIB Li_1 (GE) - TPM
ACIB Li_2 

In [69]:
# Run this cell if read_xlsx_files_to_dataframe is interrupted or raised an error. In the next run error is handled. 
# You can subset the species in another directory for the next run to increase the speed up.

# Uncomment below to continue to append to final_df

# directory = 'data_expression'
# final_df = read_xlsx_files_to_dataframe(directory, final_df)
# print(final_df)

KeyboardInterrupt: 

In [48]:
final_df

,Species,Chromosome,Region,As_1 (GE) - TPM,As_2 (GE) - TPM,As_3 (GE) - TPM,Bs_1 (GE) - TPM,Bs_2 (GE) - TPM,Bs_3 (GE) - TPM,Ctrl_1 (GE) - TPM,...,Sp_1 (GE) - TPM,Sp_2 (GE) - TPM,Sp_3 (GE) - TPM,Tm_1 (GE) - TPM,Tm_2 (GE) - TPM,Tm_3 (GE) - TPM,Vic_1 (GE) - TPM,Vic_2 (GE) - TPM,Vic_3 (GE) - TPM,Species ID
0,Achromobacter xylosoxidans SOLR10,NZ_CP025774,complement(1..400),4579.764521,2874.021275,2805.289041,3462.302060,3501.986142,3502.851956,3906.687837,...,7812.667199,8965.167866,9056.547435,3205.688636,3228.682954,3103.641082,3401.841238,3832.566805,3408.199122,ACHX
1,Achromobacter xylosoxidans SOLR10,NZ_CP025774,complement(980..2143),14.339311,17.782257,23.076558,14.063192,17.260141,12.550914,22.462166,...,4.506260,12.322023,9.714290,30.958629,32.159798,33.681898,22.375684,22.529990,19.090084,ACHX
2,Achromobacter xylosoxidans SOLR10,NZ_CP025774,complement(2164..3555),8.624839,10.448941,3.816945,6.082623,6.523412,3.794404,9.456728,...,4.306475,1.545564,3.046184,10.307187,11.364142,15.717096,6.735852,5.521991,4.961554,ACHX
3,Achromobacter xylosoxidans SOLR10,NZ_CP025774,complement(3599..4984),21.127260,38.747719,23.426751,15.679651,17.443773,17.756848,43.099754,...,4.325118,3.621928,3.824213,46.222022,37.725002,48.744498,29.690886,27.566362,33.364651,ACHX
4,Achromobacter xylosoxidans SOLR10,NZ_CP025774,complement(5049..5855),338.906377,344.525203,239.579899,222.429542,237.845060,159.168702,260.879136,...,20.427741,8.886514,10.508767,115.769842,95.915567,75.475714,208.814208,333.092076,332.654255,ACHX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105083,Streptococcus suis S10 -P1/7,NC_012925,2000927..2003506,154.651786,166.363005,158.597735,103.096554,117.077900,113.354978,153.575746,...,NaN,NaN,NaN,186.116702,195.310415,191.215538,121.440741,149.782243,68.981909,SSUIS
105084,Streptococcus suis S10 -P1/7,NC_012925,complement(2003907..2004590),51.843037,47.737511,50.150766,30.035985,27.269205,22.901773,142.767970,...,NaN,NaN,NaN,69.873527,55.227042,67.758348,94.401511,108.957265,44.032987,SSUIS
105085,Streptococcus suis S10 -P1/7,NC_012925,complement(2004615..2005094),72.526107,104.306461,90.287062,67.168967,49.922529,76.483851,292.305112,...,NaN,NaN,NaN,161.530080,151.730774,194.563257,245.576984,273.601934,308.030757,SSUIS
105086,Streptococcus suis S10 -P1/7,NC_012925,2005268..2006464,133.774758,140.802925,145.617949,191.050633,234.493519,234.235125,136.907216,...,NaN,NaN,NaN,180.352408,159.162080,162.029477,143.215049,132.640597,134.195769,SSUIS


In [49]:
final_df.to_csv('expression_final_df.csv')

In [50]:
print('Number of species: ', len(set(final_df["Species"].tolist())))
print(set(final_df["Species"].tolist()))

Number of species:  32
{'Enterococcus faecalis OG1RF', 'Mycobacterium tuberculosis H37Ra', 'Streptococcus agalactiae NEM316', 'Neisseria meningitidis serogroup C FAM18', 'Francisella tularensis subsp. holarctica FSC200', 'Streptococcus suis S10 -P1/7', 'Pseudomonas aeruginosa PAO1', 'Escherichia coli UPEC 536', 'Legionella pneumophila subsp. pneumophila Philadelphia 1', 'Streptococcus pyogenes 5448', 'Helicobacter pylori G27', 'Acinetobacter baumannii AB5075-UW', 'Shigella flexneri 5a str. M90T', 'Staphylococcus\xa0epidermidis 1457', 'Burkholderia pseudomallei K96243', 'Listeria monocytogenes EGD-e', 'Haemophilus influenzae 86-028NP', 'Helicobacter pylori J99', 'Staphylococcus\xa0aureus MSSA476', 'Achromobacter xylosoxidans SOLR10', 'Neisseria gonorrhoeae FA 1090', 'Streptococcus pneumoniae D39', 'Yersinia pseudotuberculosis YPIII', 'Vibrio cholerae O1 biovar El Tor str. N16961', 'Escherichia coli EPEC 0127:H6 E2348/69', 'Borrelia burgdorferi B31', 'Aggregatibacter actinomycetemcomitan

# upstream_sequences section (Move after you get all the species)

In [64]:
# Functions to process sequence data
def preprocess_tsv(tsv_path):
    # Read the TSV file into a pandas dataframe
    tsv_df = pd.read_csv(tsv_path, sep='\t')
    
    # Drop the .csv extension from the 'csv' column and rename it to 'Species'
    tsv_df['Species'] = tsv_df['csv'].str.replace('.csv', '', regex=False)
    
    # Rename the 'contig' column to 'Chromosome'
    tsv_df.rename(columns={'contig': 'Chromosome', 'region': 'Region'}, inplace=True)
    
    # Drop the original 'csv' column
    tsv_df.drop(columns=['csv'], inplace=True)

    # Replace specific species names
    tsv_df['Species'] = tsv_df['Species'].replace({
        'Salmonella enterica subsp. enterica serovar TyphimuriumSL1344': 'Salmonella enterica subsp. enterica serovar Typhimurium SL1344',
        'Escherichia coli EPEC 0127 H6 E2348 69': 'Escherichia coli EPEC 0127:H6 E2348/69',
        'Streptococcus suis S10 P 17': 'Streptococcus suis S10 -P1/7'
    })
    
    return tsv_df

def merge_dataframes(main_df, tsv_df):
    # Check if the two datasets have the same species
    species_diff_1 = set(tsv_df['Species']).difference(set(main_df['Species']))
    species_diff_2 = set(main_df['Species']).difference(set(tsv_df['Species']))

    if species_diff_1 or species_diff_2:
        print(f"Species present in TSV but not in main dataframe: {species_diff_1}")
        print(f"Species present in main dataframe but not in TSV: {species_diff_2}")

    # Perform an inner merge on 'Species', 'Chromosome', and 'Region' columns
    # What other merge can we do here!!
    merged_df = pd.merge(main_df, tsv_df, on=['Species', 'Chromosome', 'Region'], how='inner')
    
    return merged_df



In [65]:
# Read and preprocess the TSV file
tsv_path = 'upstream_sequences.tsv'
sequence_df = preprocess_tsv(tsv_path)



In [66]:
sequence_df

,Chromosome,Region,upstream200,Species
0,NC_002505,complement(235..402),CAGGCTCTGCAGAATACACCACCGAATACCTCTGCACTACGTTATG...,Vibrio cholerae O1 biovar El Tor str. N16961
1,NC_002505,complement(372..806),ATCTCGATGCCCTAGAGCGAGCCGCAGAGCACTTAGCGATTGGCCA...,Vibrio cholerae O1 biovar El Tor str. N16961
2,NC_002505,complement(816..2210),TGCCAGTCATGTTCACTTTCTTCTTCCTGTGGTTCCCATCAGGTCT...,Vibrio cholerae O1 biovar El Tor str. N16961
3,NC_002505,complement(2271..3896),TGGTTTATTAGTCCACTTATCGGCCCACGCTGCCGATTCACTCCTA...,Vibrio cholerae O1 biovar El Tor str. N16961
4,NC_002505,complement(3899..4156),CTTTCTCATCCTCGTTTGGGACTCGCGGTTCCTAAAAAGCAGATCA...,Vibrio cholerae O1 biovar El Tor str. N16961
...,...,...,...,...
96406,NC_002942,complement(3393934..3395274),CAGCACCAGCTGATCCAATGCAAGCTAAGGTAATGATGTTTTTACC...,Legionella pneumophila subsp. pneumophila Phil...
96407,NC_002942,complement(3395275..3396945),ATCAGTATTTTATTAGCCCTTTGATAACACCATGTTGTCGCTATTA...,Legionella pneumophila subsp. pneumophila Phil...
96408,NC_002942,complement(3396955..3397200),AATAAGTTAGGCTATGCACGCCTTGGTTTAGCATTGTCAAAAAAAA...,Legionella pneumophila subsp. pneumophila Phil...
96409,NC_002942,complement(3397167..3397355),TAAAAAGACGTCGTGCTAAAGGTCGTAAGCGTTTATCTGCCTAAGT...,Legionella pneumophila subsp. pneumophila Phil...


In [67]:
merged_df = merge_dataframes(final_df, sequence_df)

Species present in TSV but not in main dataframe: set()
Species present in main dataframe but not in TSV: {'Shigella flexneri 5a str. M90T', 'Yersinia pseudotuberculosis YPIII'}
                                 Species   Chromosome  \
0      Achromobacter xylosoxidans SOLR10  NZ_CP025774   
1      Achromobacter xylosoxidans SOLR10  NZ_CP025774   
2      Achromobacter xylosoxidans SOLR10  NZ_CP025774   
3      Achromobacter xylosoxidans SOLR10  NZ_CP025774   
4      Achromobacter xylosoxidans SOLR10  NZ_CP025774   
...                                  ...          ...   
96406       Streptococcus suis S10 -P1/7    NC_012925   
96407       Streptococcus suis S10 -P1/7    NC_012925   
96408       Streptococcus suis S10 -P1/7    NC_012925   
96409       Streptococcus suis S10 -P1/7    NC_012925   
96410       Streptococcus suis S10 -P1/7    NC_012925   

                             Region  As_1 (GE) - TPM  As_2 (GE) - TPM  \
0                complement(1..400)      4579.764521      2874.0

In [70]:
merged_df

,Species,Chromosome,Region,As_1 (GE) - TPM,As_2 (GE) - TPM,As_3 (GE) - TPM,Bs_1 (GE) - TPM,Bs_2 (GE) - TPM,Bs_3 (GE) - TPM,Ctrl_1 (GE) - TPM,...,Sp_2 (GE) - TPM,Sp_3 (GE) - TPM,Tm_1 (GE) - TPM,Tm_2 (GE) - TPM,Tm_3 (GE) - TPM,Vic_1 (GE) - TPM,Vic_2 (GE) - TPM,Vic_3 (GE) - TPM,Species ID,upstream200
0,Achromobacter xylosoxidans SOLR10,NZ_CP025774,complement(1..400),4579.764521,2874.021275,2805.289041,3462.302060,3501.986142,3502.851956,3906.687837,...,8965.167866,9056.547435,3205.688636,3228.682954,3103.641082,3401.841238,3832.566805,3408.199122,ACHX,CCTTCCAAGCTTACGACGAGGGTTCGATTCCCTTCACCCGCTCCAA...
1,Achromobacter xylosoxidans SOLR10,NZ_CP025774,complement(980..2143),14.339311,17.782257,23.076558,14.063192,17.260141,12.550914,22.462166,...,12.322023,9.714290,30.958629,32.159798,33.681898,22.375684,22.529990,19.090084,ACHX,GCATTGCATCTCGGCCGCCAGTTGCGCACGGGCACCGTCTTCATGA...
2,Achromobacter xylosoxidans SOLR10,NZ_CP025774,complement(2164..3555),8.624839,10.448941,3.816945,6.082623,6.523412,3.794404,9.456728,...,1.545564,3.046184,10.307187,11.364142,15.717096,6.735852,5.521991,4.961554,ACHX,GGAAGCGGCGCAGCGCCTGCGCCAGTCCGAAGCCGCGCGCGAGCTG...
3,Achromobacter xylosoxidans SOLR10,NZ_CP025774,complement(3599..4984),21.127260,38.747719,23.426751,15.679651,17.443773,17.756848,43.099754,...,3.621928,3.824213,46.222022,37.725002,48.744498,29.690886,27.566362,33.364651,ACHX,CGAAGGCGCGGGCGGCGTCATTACCGATTGGTCGGGGCAGCCGTTA...
4,Achromobacter xylosoxidans SOLR10,NZ_CP025774,complement(5049..5855),338.906377,344.525203,239.579899,222.429542,237.845060,159.168702,260.879136,...,8.886514,10.508767,115.769842,95.915567,75.475714,208.814208,333.092076,332.654255,ACHX,AAGGAATACACGGCAACGGCGATGAGCTGCTTTTCTGCGGACGGCG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96406,Streptococcus suis S10 -P1/7,NC_012925,2000927..2003506,154.651786,166.363005,158.597735,103.096554,117.077900,113.354978,153.575746,...,NaN,NaN,186.116702,195.310415,191.215538,121.440741,149.782243,68.981909,SSUIS,TGGCCATTACGACTGTTGCGAACAAAGTTCGCCTTATCTCCAACCT...
96407,Streptococcus suis S10 -P1/7,NC_012925,complement(2003907..2004590),51.843037,47.737511,50.150766,30.035985,27.269205,22.901773,142.767970,...,NaN,NaN,69.873527,55.227042,67.758348,94.401511,108.957265,44.032987,SSUIS,ATATAACTTTTATCATCGGTGGTAGTTTGGGTCTCGATTCTTGTAT...
96408,Streptococcus suis S10 -P1/7,NC_012925,complement(2004615..2005094),72.526107,104.306461,90.287062,67.168967,49.922529,76.483851,292.305112,...,NaN,NaN,161.530080,151.730774,194.563257,245.576984,273.601934,308.030757,SSUIS,TTTTTTCATATAAGGAATCCTTTCCATTTCTTTAATTTTCCTTAAG...
96409,Streptococcus suis S10 -P1/7,NC_012925,2005268..2006464,133.774758,140.802925,145.617949,191.050633,234.493519,234.235125,136.907216,...,NaN,NaN,180.352408,159.162080,162.029477,143.215049,132.640597,134.195769,SSUIS,TCCAACGGTAATCAATTTTATTTTCATAAAATAATTGTAACATATC...


In [71]:
merged_df.to_csv('combined_data.csv')